# Nuevos listados

En esta notebook, vamos a generar los nuevos listados de palabras...

In [1]:
%matplotlib inline

import pandas as pd

analizadas = pd.read_csv("data/listado_definitivo.csv")
analizadas.set_index(' ', inplace=True)
analizadas["candidata"] = analizadas["Palabra Candidata"]

In [2]:
palabras_ya_analizadas = analizadas[analizadas['candidata'].notna()]

print("Tenemos {} palabras analizadas".format(palabras_ya_analizadas.shape[0]))

Tenemos 5014 palabras analizadas


## Listado de Palabras

Vamos a regenerar el listado completo de palabras. Y calculemos cuánto falta trabajar si hacemos de nuevo los cálculos

In [3]:
import re

df = pd.read_csv("data/cantidades_filtradas.csv", index_col=0)
df = df[df.cantUsuariosTotal >= 2]

cant_palabras = [c for c in df.columns if re.match(r'.*Palabras$', c)]
cant_personas = [c for c in df.columns if re.match(r'.*Usuarios$', c)]

df["cant_provincias"] = (df[cant_palabras] > 0).sum(axis=1)

Calculemos el valor de la información de cada palabra.

In [4]:
from scipy.stats import entropy
import numpy as np

df["entropy_palabras"] = df[cant_palabras].apply(entropy, axis=1, raw=True)
df["entropy_personas"] = df[cant_personas].apply(entropy, axis=1, raw=True)

df["delta_palabras"] = np.log(23) - df["entropy_palabras"]
df["delta_personas"] = np.log(23) - df["entropy_personas"]

df["log_cantidad"] = np.log(1+df.cantPalabra)
df["log_personas"] = np.log(1+df.cantUsuariosTotal)

df["norm_cantidad"] = df["log_cantidad"]  / df["log_cantidad"].max()
df["norm_personas"] = df["log_personas"] / df["log_personas"].max()

df["ival_palabras"] = df.norm_cantidad * df.delta_palabras
df["ival_personas"] = df.norm_personas * df.delta_personas


df["ival"] = df["ival_palabras"] * df["ival_personas"]

df_personas = df.sort_values("ival_personas", ascending=False)
df_palabras = df.sort_values("ival_palabras", ascending=False)
df_pp = df.sort_values("ival", ascending=False)

Cambiamos ligeramente el cálculo de la normalización y del IV, contemplando el orden de personas

¿De las primeras 1000, cuántas no están en las ya analizadas?

In [5]:
def calcular_porcentaje_candidatas(palabras):
    subconj_analizado = palabras.index.intersection(palabras_ya_analizadas.index)
    candidatas = (analizadas.loc[subconj_analizado]['candidata'] != '0').sum()
    total = subconj_analizado.shape[0]
    print("Total analizadas actualmente = {}".format(total))
    
    return candidatas / total

def analizar_palabras_restantes(n, dataframes):
    total = 0
    todas_las_palabras = set()
    print("="* 80)
    print("Primeras {} palabras".format(n))
    
    for nombre, this_df in dataframes.items(): 
        print('#'*40)
        print("Métrica: {}".format(nombre))
        palabras = this_df.iloc[:n]
        
        metrica = calcular_porcentaje_candidatas(palabras)
        restantes = {p for p in palabras.index if p not in palabras_ya_analizadas.index}
        total+=len(restantes)
        todas_las_palabras = todas_las_palabras.union(restantes)
        print("Faltan etiquetar {} palabras".format(len(restantes)))
        print("Porcentaje candidatas = {}".format(metrica))
    print("Faltan etiquetar (sin repeticiones) = {}".format(len(todas_las_palabras)))

In [6]:
dataframes = {
    "Palabras": df_palabras, 
    "Personas": df_personas, 
    "PalPer": df_pp
}
analizar_palabras_restantes(1000, dataframes)
dataframes = {
    "Personas": df_personas, 
    "PalPer": df_pp
}

analizar_palabras_restantes(5000, dataframes)


Primeras 1000 palabras
########################################
Métrica: Palabras
Total analizadas actualmente = 781
Faltan etiquetar 219 palabras
Porcentaje candidatas = 0.18950064020486557
########################################
Métrica: Personas
Total analizadas actualmente = 1000
Faltan etiquetar 0 palabras
Porcentaje candidatas = 0.262
########################################
Métrica: PalPer
Total analizadas actualmente = 1000
Faltan etiquetar 0 palabras
Porcentaje candidatas = 0.225
Faltan etiquetar (sin repeticiones) = 219
Primeras 5000 palabras
########################################
Métrica: Personas
Total analizadas actualmente = 4347
Faltan etiquetar 653 palabras
Porcentaje candidatas = 0.4200598113641592
########################################
Métrica: PalPer
Total analizadas actualmente = 3943
Faltan etiquetar 1057 palabras
Porcentaje candidatas = 0.3814354552371291
Faltan etiquetar (sin repeticiones) = 1235


## Usando cálculo viejo


In [7]:
from contrastes.information_value import simulated_shuffled_entropy

In [8]:
df = pd.read_csv("data/cantidades_filtradas.csv", index_col=0)
df = df[df.cantUsuariosTotal >= 2]

cant_palabras = [c for c in df.columns if re.match(r'.*Palabras$', c)]
cant_personas = [c for c in df.columns if re.match(r'.*Usuarios$', c)]

df["cant_provincias"] = (df[cant_palabras] > 0).sum(axis=1)

fn = lambda ws: simulated_shuffled_entropy(int(ws), len(cant_palabras))
df["sh_entropy_palabras"] = df.cantPalabra.apply(fn)
df["sh_entropy_personas"] = df.cantUsuariosTotal.apply(fn)


In [9]:
df["entropy_palabras"] = df[cant_palabras].apply(entropy, axis=1, raw=True)
df["entropy_personas"] = df[cant_personas].apply(entropy, axis=1, raw=True)

df["delta_palabras"] = df["sh_entropy_palabras"] - df["entropy_palabras"]
df["delta_personas"] = df["sh_entropy_personas"] - df["entropy_personas"]

df["log_cantidad"] = np.log(df.cantPalabra)
df["log_personas"] = np.log(df.cantUsuariosTotal)

df["norm_cantidad"] = (df["log_cantidad"] - df["log_cantidad"].min()) \
    / (df["log_cantidad"].max() - df["log_cantidad"].min())
df["norm_personas"] = (df["log_personas"] - df["log_personas"].min()) \
    / (df["log_personas"].max() - df["log_personas"].min())
    
df["ival_palabras"] = df.norm_cantidad * df.delta_palabras
df["ival_personas"] = df.norm_personas * df.delta_personas


df["ival"] = df["ival_palabras"] * df["ival_personas"]

df_personas = df.sort_values("ival_personas", ascending=False)
df_palabras = df.sort_values("ival_palabras", ascending=False)
df_pp = df.sort_values("ival", ascending=False)

In [10]:
dataframes = {
    "Palabras": df_palabras, 
    "Personas": df_personas, 
    "PalPer": df_pp
}
analizar_palabras_restantes(1000, dataframes)
dataframes = {
    "Personas": df_personas, 
    "PalPer": df_pp
}

analizar_palabras_restantes(5000, dataframes)


Primeras 1000 palabras
########################################
Métrica: Palabras
Total analizadas actualmente = 790
Faltan etiquetar 210 palabras
Porcentaje candidatas = 0.2708860759493671
########################################
Métrica: Personas
Total analizadas actualmente = 1000
Faltan etiquetar 0 palabras
Porcentaje candidatas = 0.293
########################################
Métrica: PalPer
Total analizadas actualmente = 664
Faltan etiquetar 336 palabras
Porcentaje candidatas = 0.2560240963855422
Faltan etiquetar (sin repeticiones) = 546
Primeras 5000 palabras
########################################
Métrica: Personas
Total analizadas actualmente = 4228
Faltan etiquetar 772 palabras
Porcentaje candidatas = 0.4231315042573321
########################################
Métrica: PalPer
Total analizadas actualmente = 1242
Faltan etiquetar 3758 palabras
Porcentaje candidatas = 0.29790660225442833
Faltan etiquetar (sin repeticiones) = 4530
